In [ ]:

!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c conda-forge glpk

!pip install pyomo



⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...
Solving environment: | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.4



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.7.22  |       hbcca054_0         146 KB  conda-forge
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    glpk

In [ ]:
from pyomo.environ import *

### Code for abstract model

In [ ]:
# Create an instance of the model
model = AbstractModel()

# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N) #within parameter is a set used for validation (ie network arcs are in model.N*model.N matrix)

# Source node
model.s = Param(within=model.N)
# Sink node
model.t = Param(within=model.N)
# Flow capacity limits
model.u = Param(model.A) #sets up a parameter c that is indexed by model.A
# Arc interdiction costs
model.c = Param(model.A)
# Interdiction budget
model.b = Param(default=1)

# Decision variable: interdict this arc?
model.y = Var(model.A, within=Binary)
model.theta = Var(model.A, within=NonNegativeReals)
model.pi = Var(model.N, within=NonNegativeReals)

# Objective: minimize the capacity of the min cut
# Define objective rule
def total_rule(model):
    return sum((model.u[i,j]*model.theta[i,j]) for (i, j) in model.A)
# set objective
model.total = Objective(rule=total_rule, sense=minimize)

# Constraint
def budget_rule(model):
    return sum(model.c[i,j]*model.y[i,j] for (i,j) in model.A) <= model.b
model.budget = Constraint(rule=budget_rule)

# Constraint
def rule_one(model, i, j):
    if i != value(model.s):
        return Constraint.Skip
    return model.pi[j] +  model.theta[model.s, j] >= 1 - model.y[model.s, j]
model.rule1 = Constraint(model.A, rule=rule_one)

# Constraint
def rule_two(model, i, j):
    if i == value(model.s) or j == value(model.t):
        return Constraint.Skip
    return model.pi[j] - model.pi[i] + model.theta[i,j] >= -model.y[i,j]
model.rule2 = Constraint(model.A, rule=rule_two)

# Constraint
def rule_three(model, i, j):
    if j != value(model.t):
        return Constraint.Skip
    return -model.pi[i] + model.theta[i,model.t] >= -model.y[i,model.t]
model.rule3 = Constraint(model.A, rule=rule_three)


### Initialize model with data

In [ ]:
data = DataPortal()
data.load(filename='maxflow_interdiction.dat', model=model)
instance = model.create_instance(data)

### Print model

In [ ]:
instance.pprint()

### Results

In [ ]:
opt = SolverFactory('glpk')
opt.solve(instance).write()


### Variables

In [ ]:
instance.y.pprint()
instance.theta.pprint()
instance.pi.pprint()

In [ ]:
min_cut_capacity = 0

for (i,j) in instance.A:
  min_cut_capacity += (instance.theta[i,j].value * instance.u[i,j])

print("Interdicted arc(s): ", end=" ")
for (i,j) in instance.A:
  if instance.y[i,j].value == 1:
    print('(',i,j,')', end=" ")
print()

print("Max flow after interdiction: ", min_cut_capacity)